In [1]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time
from pathlib                        import Path
from PIL                            import Image

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
import gc
import glob
from skimage                        import img_as_ubyte
from skimage                        import io
import shutil
tf.keras.backend.clear_session()

2025-05-09 16:53:51.685124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746824031.698847   36762 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746824031.702826   36762 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746824031.713048   36762 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746824031.713065   36762 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746824031.713066   36762 computation_placer.cc:177] computation placer alr

### Working with ImageDataGenerator

In [2]:
general_directory ='/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5'

def dataset(directory,batch_size=32):
    data = keras.utils.image_dataset_from_directory(directory,
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)
    return data

path_1= general_directory + '/geo_masked' + '/train'
path_2= general_directory + '/geo_masked' + '/test'
path_3= general_directory + '/geo_masked' + '/valid'

path_4= general_directory + '/mag_masked' + '/train'
path_5= general_directory + '/mag_masked' + '/test'
path_6= general_directory + '/mag_masked' + '/valid'

path_7= general_directory + '/vx_masked' + '/train'
path_8= general_directory + '/vx_masked' + '/test'
path_9= general_directory + '/vx_masked' + '/valid'

path_10= general_directory + '/vy_masked' + '/train'
path_11= general_directory + '/vy_masked' + '/test'
path_12= general_directory + '/vy_masked' + '/valid'

path_13= general_directory + '/p_masked' + '/train'
path_14= general_directory + '/p_masked' + '/test'
path_15= general_directory + '/p_masked' + '/valid'
    

In [3]:
g_train = dataset(path_1)
g_test  = dataset(path_2)
g_valid = dataset(path_3)
print('******************')
v_train = dataset(path_4)
v_test  = dataset(path_5)
v_valid = dataset(path_6)
print('******************')
vx_train = dataset(path_7)
vx_test  = dataset(path_8)
vx_valid = dataset(path_9)
print('******************')
vy_train = dataset(path_10)
vy_test  = dataset(path_11)
vy_valid = dataset(path_12)
print('******************')
p_train = dataset(path_13)
p_test = dataset(path_14)
p_valid  = dataset(path_15)

Found 14000 files.
Found 2000 files.
Found 4000 files.


I0000 00:00:1746824041.601121   36762 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 83 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


******************
Found 14000 files.
Found 2000 files.
Found 4000 files.
******************
Found 14000 files.
Found 2000 files.
Found 4000 files.
******************
Found 14000 files.
Found 2000 files.
Found 4000 files.
******************
Found 14000 files.
Found 2000 files.
Found 4000 files.


In [4]:
# combination of output datasets
output_train = tf.data.Dataset.zip((v_train,vx_train,vy_train,p_train))
output_test = tf.data.Dataset.zip((v_test,vx_test,vy_test,p_test))
output_valid = tf.data.Dataset.zip((v_valid,vx_valid,vy_valid,p_valid))

# combination of input datasets with output datasets
final_train = tf.data.Dataset.zip((g_train,output_train))
final_test = tf.data.Dataset.zip((g_test,output_test))
final_valid = tf.data.Dataset.zip((g_valid,output_valid))

### Standarize the data

In [5]:
# Standardizing values to be in the [0, 1] range by using tf.keras.layers.Rescaling:
normalization_layer = tf.keras.layers.Rescaling(1./255)

final_train_normalized = final_train.map(lambda x, y: (normalization_layer(x), y))
final_test_normalized = final_test.map(lambda x, y: (normalization_layer(x), y))
final_valid_normalized = final_valid.map(lambda x, y: (normalization_layer(x), y))


### Hiperparámeters

In [6]:
num_epochs = 5
patience=5  # How long to wait after last time validation loss improved
LR=0.001
# Model name
model="U-Net"
plot_name='U-Net_Model.png'
save_in='/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5'
# image dimensions
img_width             =  256   # 739   G:737
img_height            =  128   # 185
channel               =  1

### Architecture

In [7]:

image_input = Input((img_height, img_width, channel))

# 64 3x3 convolutions followed by a batchnormalization and max pool
conv1 = Conv2D(64, (3, 3), padding='same')(image_input)
conv1= BatchNormalization()(conv1)
conv1 = Activation("relu")(conv1)
conv1 = Conv2D(64, (3, 3), padding='same')(conv1)
conv1= BatchNormalization()(conv1)
conv1 = Activation("relu")(conv1)
pool1=MaxPooling2D((2,2))(conv1)

# 128 3x3 convolutions followed by a batchnormalization and max pool
conv2 = Conv2D(128, (3, 3), padding='same')(pool1)
conv2= BatchNormalization()(conv2)
conv2 = Activation("relu")(conv2)
conv2 = Conv2D(128, (3, 3), padding='same')(conv2)
conv2= BatchNormalization()(conv2)
conv2 = Activation("relu")(conv2)
pool2=MaxPooling2D((2,2))(conv2)

# 256 3x3 convolutions followed by a batchnormalization and max pool
conv3 = Conv2D(256, (3, 3), padding='same')(pool2)
conv3= BatchNormalization()(conv3)
conv3 = Activation("relu")(conv3)
conv3 = Conv2D(256, (3, 3), padding='same')(conv3)
conv3= BatchNormalization()(conv3)
conv3 = Activation("relu")(conv3)
pool3=MaxPooling2D((2,2))(conv3)

# 512 3x3 convolutions followed by a batchnormalization and max pool
conv4 = Conv2D(512, (3, 3), padding='same')(pool3)
conv4= BatchNormalization()(conv4)
conv4 = Activation("relu")(conv4)
conv4 = Conv2D(512, (3, 3), padding='same')(conv4)
conv4= BatchNormalization()(conv4)
conv4= Activation("relu")(conv4)
pool4=MaxPooling2D((2,2))(conv4)

# 1024 Bottleneck  3x3 convolutions
conv5 = Conv2D(1024, (3,3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(1024, (3,3), activation='relu', padding='same')(conv5)

# 512 3x3 transpose convolution and concate
conv6=Conv2DTranspose(512,(2,2),strides=(2, 2),padding="same")(conv5) # remark: if it's used padding=valid, apply copy and crop the skip features
up6=concatenate([conv6,conv4])
conv6=Conv2D(512, (3, 3), activation="relu", padding="same")(up6)
conv6=Conv2D(512, (3, 3), activation="relu", padding="same")(conv6)

# 216 3x3 transpose convolution and concate
conv7=Conv2DTranspose(216,(2,2),strides=(2, 2),padding="same")(conv6) 
up7=concatenate([conv7,conv3])
conv7=Conv2D(216, (3, 3), activation="relu", padding="same")(up7)
conv7=Conv2D(216, (3, 3), activation="relu", padding="same")(conv7)

# 128 3x3 transpose convolution and concate
conv8=Conv2DTranspose(128,(2,2),strides=(2, 2),padding="same")(conv7) 
up8=concatenate([conv8,conv3])
conv8=Conv2D(128, (3, 3), activation="relu", padding="same")(up8)
conv8=Conv2D(128, (3, 3), activation="relu", padding="same")(conv8)

# 64 3x3 transpose convolution and concate
conv9=Conv2DTranspose(64,(2,2),strides=(2, 2),padding="same")(conv8) 
up9=concatenate([conv9,conv2])
conv9=Conv2D(64, (3, 3), activation="relu", padding="same")(up9)
conv9=Conv2D(64, (3, 3), activation="relu", padding="same")(conv9)

# **********************************************************************************************
# Construc  model
# final 1x1 convolutions to get to the correct depth dim (num_classes=4 for v, v-x, v-y, p)
output1 = Conv2D(1,(1,1),activation="relu",name='output_v', padding="same")(conv9)

#output1 = Conv2D(1,(1,1),activation="relu", padding="same")(conv5)
#output1_masked = keras.layers.Multiply(name='output_v')([image_input, output1])

output2 = Conv2D(1,(1,1),activation="relu",name='output_vx', padding="same")(conv9)

#output2 = Conv2D(1,(1,1),activation="relu", padding="same")(conv5)
#output2_masked = keras.layers.Multiply(name='output_vx')([image_input, output2])

output3 = Conv2D(1,(1,1),activation="relu",name='output_vy', padding="same")(conv9)

#output3 = Conv2D(1,(1,1),activation="relu", padding="same")(conv5)
#output3_masked = keras.layers.Multiply(name='output_vy')([image_input, output3])

output4 = Conv2D(1,(1,1),activation="relu",name='output_p', padding="same")(conv9)

#output4 = Conv2D(1,(1,1),activation="relu", padding="same")(conv5)
#output4_masked = keras.layers.Multiply(name='output_p')([image_input, output4])

#model = keras.Model(inputs=image_input, outputs=[output1_masked, output2_masked, output3_masked, output4_masked],name= 'U-Net')
model = keras.Model(inputs=image_input, outputs=[output1, output2, output3, output4],name= 'U-Net')

model.summary()


2025-05-09 16:54:42.502667: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 18.00MiB (rounded to 18874368)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-05-09 16:54:42.502714: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-05-09 16:54:42.502737: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 37, Chunks in use: 37. 9.2KiB allocated for chunks. 9.2KiB in use in bin. 2.6KiB client-requested in use in bin.
2025-05-09 16:54:42.502750: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 10, Chunks in use: 10. 5.0KiB allocated for chunks. 5.0KiB in use in bin. 5.0KiB client-requested in use in bin.
2025-05-09 16:

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 

In [ ]:
model.compile(loss=['mse', 'mse', 'mse', 'mse'], optimizer='RMSprop', metrics=["mae","mae","mae","mae"])

print("Starting trainig")

history = model.fit(final_train,epochs=num_epochs,validation_data = final_valid)
   
model.compile(optimizer='RMSprop', 
              loss = {  'output_v': 'mean_squared_error' ,
                        'output_vx': 'mean_squared_error' , 
                        'output_vy': 'mean_squared_error',
                        'output_p': 'mean_squared_error'},
              
              metrics={ 'output_v': ['mae'],
                        'output_vx': ['mae'],
                        'output_vy': ['mae'],
                        'output_p': ['mae'] })

In [ ]:
results = model.evaluate(final_test) 
print("Evaluation results:")
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")

In [ ]:
for x_test,y_true in final_test.take(1):
    y_pred = model.predict(x_test)
    for i in range(4):
        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        axs[0].imshow(x_test[0],cmap='gray')
        axs[0].set_title("Input")

        axs[1].imshow(y_true[i][0],cmap='gray')
        axs[1].set_title(f"True Output {i+1}")

        axs[2].imshow(y_pred[i][0],cmap='gray')
        axs[2].set_title(f"Pred Output {i+1}")
        plt.show()
        

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    return x

def encoder_block(x, filters):
    c = conv_block(x, filters)
    p = layers.MaxPooling2D((2, 2))(c)
    return c, p

def decoder_block(x, skip, filters):
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x

def unet_with_3_decoders(input_shape=(128, 128, 1)):
    inputs = tf.keras.Input(input_shape)

    # Encoder
    c1, p1 = encoder_block(inputs, 16)
    c2, p2 = encoder_block(p1, 32)
    c3, p3 = encoder_block(p2, 64)

    # Bottleneck
    b = conv_block(p3, 128)

    # Decoder 1 (e.g., salida u)
    d1_1 = decoder_block(b, c3, 64)
    d1_2 = decoder_block(d1_1, c2, 32)
    d1_3 = decoder_block(d1_2, c1, 16)
    out_u = layers.Conv2D(1, 1, activation='linear', name='u_output')(d1_3)

    # Decoder 2 (e.g., salida v)
    d2_1 = decoder_block(b, c3, 64)
    d2_2 = decoder_block(d2_1, c2, 32)
    d2_3 = decoder_block(d2_2, c1, 16)
    out_v = layers.Conv2D(1, 1, activation='linear', name='v_output')(d2_3)

    # Decoder 3 (e.g., salida presión)
    d3_1 = decoder_block(b, c3, 64)
    d3_2 = decoder_block(d3_1, c2, 32)
    d3_3 = decoder_block(d3_2, c1, 16)
    out_p = layers.Conv2D(1, 1, activation='linear', name='p_output')(d3_3)

    # Modelo final
    model = Model(inputs, [out_u, out_v, out_p])
    return model
